In [2]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import config

In [62]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import pandas as pd
import json
from tqdm import tqdm

In [41]:
with open("../../data/player_summaries.json", "r", encoding="utf-8") as f:
    players_summaries = json.load(f)
    
df_summaries = pd.DataFrame([
    {'player': player.split('(')[0].strip(), 'team': player.split('(')[1].replace(')', '').strip(), 'summary': summary} for player, summary in players_summaries.items()
])
df_summaries = df_summaries.reset_index() 
df_summaries.head()

,index,player,team,summary
0,0,Ben White,Arsenal,"Ben White, défenseur de 26 ans évoluant à Arse..."
1,1,Bukayo Saka,Arsenal,"Bukayo Saka, à seulement 22 ans, s'affirme com..."
2,2,David Raya,Arsenal,"David Raya, gardien de but d'Arsenal, se disti..."
3,3,Declan Rice,Arsenal,"Declan Rice, milieu de terrain d'Arsenal, se d..."
4,4,Ethan Nwaneri,Arsenal,Ethan Nwaneri est un jeune talent prometteur d...


In [42]:
df_players = pd.read_csv('../../data/players_stats.csv')
df_players = df_players.reset_index(drop=True)
df_players.head()

,Unnamed: 0,index,league,season,team,player,nation__standard,pos__standard,age__standard,born__standard,...,Performance_Crs_misc,Performance_Int_misc,Performance_TklW_misc,Performance_PKwon_misc,Performance_PKcon_misc,Performance_OG_misc,Performance_Recov_misc,Aerial Duels_Won_misc,Aerial Duels_Lost_misc,Aerial Duels_Won%_misc
0,0,0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,26.0,1997.0,...,17,16,14,0,0,0,37,18,9,66.7
1,1,1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"FW,MF",22.0,2001.0,...,117,3,15,1,0,0,70,10,20,33.3
2,2,2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28.0,1995.0,...,0,2,0,0,1,0,41,13,0,100.0
3,3,3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25.0,1999.0,...,164,25,25,0,0,0,156,37,25,59.7
4,4,4,ENG-Premier League,2425,Arsenal,Ethan Nwaneri,ENG,"FW,MF",17.0,2007.0,...,54,2,7,0,0,0,34,4,8,33.3


In [58]:
df_players_summaries = df_summaries.merge(df_players, how='left', on=['player', 'team'])
df_players_summaries = df_players_summaries = df_players_summaries[[
    'league',
    'season',
    'player',
    'team',
    'pos__standard',
    'summary'
]]

df_players_summaries.rename(columns={'pos__standard': 'position'}, inplace=True)
df_players_summaries.head()

,league,season,player,team,position,summary
0,ENG-Premier League,2425,Ben White,Arsenal,DF,"Ben White, défenseur de 26 ans évoluant à Arse..."
1,ENG-Premier League,2425,Bukayo Saka,Arsenal,"FW,MF","Bukayo Saka, à seulement 22 ans, s'affirme com..."
2,ENG-Premier League,2425,David Raya,Arsenal,GK,"David Raya, gardien de but d'Arsenal, se disti..."
3,ENG-Premier League,2425,Declan Rice,Arsenal,MF,"Declan Rice, milieu de terrain d'Arsenal, se d..."
4,ENG-Premier League,2425,Ethan Nwaneri,Arsenal,"FW,MF",Ethan Nwaneri est un jeune talent prometteur d...


In [66]:
embedding_model = SentenceTransformer("BAAI/bge-m3")
qdrant_client = QdrantClient(
    host='localhost', 
    port=6333,
    timeout=30.0
)

In [ ]:
collection_name = 'ragscout_players'

if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=1024,
            distance=Distance.COSINE
        )
    )

In [74]:
points = []
for idx, row in tqdm(df_players_summaries.iterrows()):
    embedding = embedding_model.encode(row['summary']).tolist()
    
    metadata = {
        'season': row['season'],
        'player': row['player'],
        'league': row['league'],
        'team': row['team'],
        'position': row['position'],
        'summary': row['summary']
    }
    
    point = PointStruct(
        id=idx,
        vector=embedding,
        payload=metadata
    )
    
    points.append(point)

0it [00:00, ?it/s]/Users/florian/.pyenv/versions/scout_rag_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
1735it [02:47, 10.34it/s]


In [75]:
BATCH_SIZE = 100

for i in range(0, len(points), BATCH_SIZE):
    batch = points[i:i + BATCH_SIZE]
    qdrant_client.upsert(
        collection_name=collection_name, 
        points=batch
    )
    
    print(f"✅ {min(i + BATCH_SIZE, len(points))} / {len(points)} envoyé")

print(f"✅ {len(points)} joueurs insérés dans Qdrant avec succès !")

✅ 100 / 1735 envoyé
✅ 200 / 1735 envoyé
✅ 300 / 1735 envoyé
✅ 400 / 1735 envoyé
✅ 500 / 1735 envoyé
✅ 600 / 1735 envoyé
✅ 700 / 1735 envoyé
✅ 800 / 1735 envoyé
✅ 900 / 1735 envoyé
✅ 1000 / 1735 envoyé
✅ 1100 / 1735 envoyé
✅ 1200 / 1735 envoyé
✅ 1300 / 1735 envoyé
✅ 1400 / 1735 envoyé
✅ 1500 / 1735 envoyé
✅ 1600 / 1735 envoyé
✅ 1700 / 1735 envoyé
✅ 1735 / 1735 envoyé
✅ 1735 joueurs insérés dans Qdrant avec succès !


In [78]:
query = 'Ailier dynamique, fort impact de buts et passe décisives, fort à la finition. Rapide, dribbleur, avec pas mal de passes clés'
query_vector = embedding_model.encode(query).tolist()

results = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=5
)

for hit in results:
    print(f"🔹 {hit.payload['player']} ({hit.payload['team']})")
    print(f"Résumé : {hit.payload['summary']}")
    print(f"Score : {hit.score:.3f}")
    print("---")

/Users/florian/.pyenv/versions/scout_rag_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


🔹 Ivan Ilić (Torino)
Résumé : Ivan Ilić, milieu de terrain de 23 ans évoluant au Torino, se distingue par un style de jeu axé sur la possession et la distribution. Avec une précision de passe de 83,4%, il se révèle être un passeur fiable, surtout dans les passes courtes et moyennes. Son impact offensif est modéré, avec seulement un but et aucune passe décisive à son actif cette saison, bien que ses statistiques d'attentes de buts (xG) et d'assists (xA) laissent entrevoir un potentiel d'amélioration avec des valeurs respectives de 0,9 et 1,1. Défensivement, il affiche une solidité honorable, avec des interceptions et des tacles qui témoignent de son engagement sur le terrain, même si son taux de réussite dans les duels est à améliorer (25% de réussite). En somme, Ilić est un joueur prometteur qui mérite d'être suivi, mais il pourrait bénéficier de davantage de présence dans la surface adverse pour devenir un véritable atout offensif pour son équipe.
Score : 0.655
---
🔹 Ruben Aguilar (Le

/var/folders/x9/t16jfwq91z735wm10ntxvh780000gq/T/ipykernel_40582/2488300388.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(
